# Build parse data

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
import pickle
import sys
sys.path.append('../../vietadminunits')
from utils import to_alphanumeric, to_key

## Functions

In [ ]:
def add_province_key(df_province, province_english, province_key):
    if not df_province[df_province.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif df_province[(df_province.province_english==province_english) & (df_province.province_key==province_key)].shape[0]:
        raise ValueError(f'{province_key} is exist in province_key')
    
    df_new = df_province.loc[df_province.province_english==province_english].head(1)
    df_new['province_key'] = province_key
    df_province = pd.concat([df_province, df_new])
    return df_province

In [ ]:
def add_district_key(df_district, province_english, district_english, district_key):
    if not df_district[df_district.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    elif not df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english)].shape[0]:
        raise ValueError(f'{district_english} is not exist in district_english of {province_english}')
    elif df_district[(df_district.province_english==province_english) & (df_district.district_english==district_english) & (df_district.district_key==district_key)].shape[0]:
        raise ValueError(f'{district_key} is exist in district_key of {province_english}, {district_english}')
    
    df_new = df_district.loc[(df_district.province_english==province_english) & (df_district.district_english==district_english)].head(1)
    df_new['district_key'] = district_key
    df_district = pd.concat([df_district, df_new])
    return df_district

In [ ]:
def add_ward_key(df_ward, province_english, district_english, ward_english, ward_key):
    if not df_ward[df_ward.province_english==province_english].shape[0]:
        raise ValueError(f'{province_english} is not exist in province_english')
    
    elif not df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english)].shape[0]:
        raise ValueError(f'{district_english} is not exist in district_english of {province_english}')
    
    elif not df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english)].shape[0]:
        raise ValueError(f'{ward_english} is not exist in ward_english of {province_english}, {district_english}')
    
    elif df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english) & (df_ward.ward_key==ward_key)].shape[0]:
        raise ValueError(f'{ward_key} is exist in ward_key of {province_english}, {district_english}, {ward_english}')
    
    df_new = df_ward.loc[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_english==ward_english)].head(1)
    df_new['ward_key'] = ward_key
    df_ward = pd.concat([df_ward, df_new])
    return df_ward

## Building

In [ ]:
df = pd.read_csv('../../data/output/vietnam_administrative_units.csv')

In [ ]:
df.head()

In [ ]:
df.shape

### Create keyword columns and fillna NULL columns

In [ ]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['short_district'].apply(to_key, args=(2,))
df['ward_key'] = df['short_ward'].apply(to_key, args=(3,))
df['district_level_english'].fillna('', inplace=True)
df['ward_level_english'].fillna('', inplace=True)

### Create keyword lists that are duplicated district_keyword

In [ ]:
district_count = df[['province', 'long_district', 'district_key']].drop_duplicates()[['province', 'district_key']].value_counts().reset_index()
duplicated_districts = district_count[district_count['count'] > 1]['district_key'].tolist()

In [ ]:
df_duplicated_districts = df[df.district_key.isin(duplicated_districts)][['long_province', 'long_district' ,'province_key','district_key']].drop_duplicates()

In [ ]:
df_duplicated_districts

In [ ]:
# Idea: Use their ward_key if possible
for district_key in duplicated_districts:
    long_districts = df[df.district_key==district_key]['long_district'].unique().tolist()
    a = []
    b = []
    long_district_a = long_districts[0]
    long_district_b = long_districts[1]
    ward_key_a = df[df.long_district==long_district_a]['ward_key'].unique().tolist()
    ward_key_b = df[df.long_district==long_district_b]['ward_key'].unique().tolist()
    common = list(set(a) & set(b))
    if common:
        print(district_key, 'has duplicated ward_key')

In [ ]:
# Pickle
# Use Google Trend and search to decide level
duplicated_district_province_keys = df_duplicated_districts['province_key'].unique().tolist()

duplicated_district_keys = {
    'kyanh': {'default':'Town'},
    'cailay': {'default':'Town'},
    'duyenhai': {'default':'District'},
    'caolanh': {'default':'City'},
    'hongngu': {'default':'City'},
    'longmy': {'default':'District'}
}

for district_key in duplicated_districts:
    df_temp = df[df.district_key == district_key]
    level_data = {}
    levels = df_temp.district_level_english.unique().tolist()
    for district_level_english in levels:
        ward_keys = df_temp[df_temp.district_level_english==district_level_english]['ward_key'].unique().tolist()
        level_data[district_level_english] = ward_keys
        duplicated_district_keys[district_key]['levels'] = level_data

In [ ]:
print('Provinces:', duplicated_district_province_keys)
print('Districts:', duplicated_district_keys)

In [ ]:
# Pickle
half_district_keys = {'mocay': {'mocaynam': ['mocay',
   'dinhthuy',
   'daphuochoi',
   'tanhoi',
   'phuochiep',
   'binhkhanh',
   'anthanh',
   'andinh',
   'thanhthoib',
   'tantrung',
   'anthoi',
   'thanhthoia',
   'minhduc',
   'ngaidang',
   'camson',
   'huongmy'],
  'mocaybac': ['phumy',
   'hungkhanhtrunga',
   'thanhtan',
   'thanhngai',
   'tanphutay',
   'phuocmytrung',
   'tanthanhbinh',
   'thanhan',
   'hoaloc',
   'tanthanhtay',
   'tanbinh',
   'nhuanphutan',
   'khanhthanhtan']},
 'hamthuan': {'hamthuanbac': ['malam',
   'phulong',
   'lada',
   'dongtien',
   'thuanhoa',
   'donggiang',
   'hamphu',
   'hongliem',
   'thuanminh',
   'hongson',
   'hamtri',
   'hamduc',
   'hamliem',
   'hamchinh',
   'hamhiep',
   'hamthang',
   'dami'],
  'hamthuannam': ['thuannam',
   'mythanh',
   'hamcan',
   'muongman',
   'hamthanh',
   'hamkiem',
   'hamcuong',
   'hammy',
   'tanlap',
   'hamminh',
   'thuanquy',
   'tanthuan',
   'tanthanh']},
 'tuliem': {'namtuliem': ['caudien',
   'xuanphuong',
   'phuongcanh',
   'mydinh1',
   'mydinh2',
   'taymo',
   'metri',
   'phudo',
   'daimo',
   'trungvan'],
  'bactuliem': ['thuongcat',
   'lienmac',
   'dongngac',
   'ducthang',
   'thuyphuong',
   'taytuu',
   'xuandinh',
   'xuantao',
   'minhkhai',
   'conhue1',
   'conhue2',
   'phudien',
   'phucdien']},
 'tramy': {'bactramy': ['tramy',
   'trason',
   'trakot',
   'tranu',
   'tradong',
   'traduong',
   'tragiang',
   'trabui',
   'tradoc',
   'tratan',
   'tragiac',
   'tragiap',
   'traka'],
  'namtramy': ['traleng',
   'tradon',
   'tratap',
   'tramai',
   'tracang',
   'tralinh',
   'tranam',
   'travan',
   'travinh']},
 'gocong': {'gocong': ['phuong2',
   'phuong1',
   'phuong5',
   'longhung',
   'longthuan',
   'longchanh',
   'longhoa',
   'binhdong',
   'binhxuan',
   'tantrung'],
  'gocongtay': ['vinhbinh',
   'dongson',
   'binhphu',
   'dongthanh',
   'thanhcong',
   'binhnhi',
   'yenluong',
   'thanhtri',
   'thanhnhut',
   'longvinh',
   'binhtan',
   'vinhhuu',
   'longbinh'],
  'gocongdong': ['tanhoa',
   'tanghoa',
   'tanphuoc',
   'giathuan',
   'vamlang',
   'tantay',
   'kiengphuoc',
   'tandong',
   'binhan',
   'tandien',
   'binhnghi',
   'phuoctrung',
   'tanthanh']}}

### Create keyword lists that are duplicated ward_keyword

In [ ]:
ward_count = df[['long_province', 'long_district', 'short_ward']].value_counts().reset_index()
df_duplicated_wards = ward_count[ward_count['count'] > 1].copy()
df_duplicated_wards.sort_values(by=['long_province', 'long_district', 'short_ward'], inplace=True)

In [ ]:
df_duplicated_wards = df[(df.long_province.isin(df_duplicated_wards.long_province)) & (df.long_district.isin(df_duplicated_wards.long_district)) & (df.short_ward.isin(df_duplicated_wards.short_ward))][['province', 'long_district', 'ward','ward_level_english', 'district_key', 'ward_key']]

In [ ]:
df_duplicated_wards

Following Google Trend, "xã ..." is higher "thị trấn ..." significant. But "phường ..." is higher than "xã ..." and we have only one ward in the list.

In [ ]:
# Pickle
duplicated_ward_keys = df_duplicated_wards.ward_key.unique().tolist()
duplicated_ward_district_keys = df_duplicated_wards.district_key.unique().tolist()

new_data = {}

for ward_key in duplicated_ward_keys:
    levels = df_duplicated_wards[df_duplicated_wards.ward_key == ward_key]['ward_level_english'].tolist()
    if 'Ward' in levels:
        ward_level = 'Ward'
    elif 'Commune' in levels:
        ward_level = 'Commune'
    else:
        ward_level = 'Town'
    new_data[ward_key] = ward_level
    
duplicated_ward_keys = new_data.copy()

In [ ]:
print('Districts:' ,duplicated_ward_district_keys)
print('Wards:', duplicated_ward_keys)

### Sort province_key to prioritize some provinces
- Some province has districts, or wards that are the same keyword with another province.
- We need to run module_testing to check whether any address is wrong > Use ChatGPT to sort keywords > Add manually if we found new wrong keyword.

For example:
- 'Huyện Quang Bình, Tỉnh Hà Giang' -> quangbinh
- 'Huyện Phù Yên, Tỉnh Sơn La' -> phuyen
- 'Huyện Văn Giang, Tỉnh Hưng Yên' -> angiang
- 'Huyện Quảng Ninh, Tỉnh Quảng Bình' -> quangninh
- Bac Lieu, Hoa Binh District -> hoabinh

In [ ]:
province_keys = df.province_key.unique().tolist()

In [ ]:
def find_province_key_match(text):
    for province_key in province_keys:
        if province_key in text:
            return province_key

In [ ]:
match_district = df[df.district_key.str.contains('|'.join(province_keys))][['province', 'district', 'province_key','district_key']].drop_duplicates()

In [ ]:
match_ward = df[(~df.ward.isna()) & (df.ward_key.str.contains('|'.join(province_keys)))][['province', 'ward', 'province_key','ward_key']].drop_duplicates()

In [ ]:
match_district['match_province_key'] = match_district['district_key'].apply(find_province_key_match)

In [ ]:
match_ward['match_province_key'] = match_ward['ward_key'].apply(find_province_key_match)

In [ ]:
match_district = match_district[match_district.province_key != match_district.match_province_key]

In [ ]:
match_district

In [ ]:
match_ward = match_ward[match_ward.province_key != match_ward.match_province_key]

In [ ]:
match_ward

In [ ]:
# Should not use list(set()) because it will re-order
province_order = []

for row in match_district.itertuples():
    if row.province_key not in province_order:
        province_order.append(row.province_key)
    if row.match_province_key not in province_order:
        province_order.append(row.match_province_key)
        
for row in match_ward.itertuples():
    if row.province_key not in province_order:
        province_order.append(row.province_key)
    if row.match_province_key not in province_order:
        province_order.append(row.match_province_key)

In [ ]:
province_key_order = {}
for index, province_key in enumerate(province_order):
    province_key_order[province_key] = index + 1
    

In [ ]:
print(province_key_order)

In [ ]:
df['province_key_order'] = df['province_key'].map(province_key_order).fillna(0)

df.sort_values(by='province_key_order', inplace=True)

### Add alias province_keyword
- We should not add "hn" because it will cause many wrong matches. I will replace `\bhn\b` to `\bha noi\b` of the address, this step is in `parse.py`

In [ ]:
df_province = df[[col for col in df.columns if 'province' in col]].drop_duplicates()

province_alias_keys = [
    ('Ho Chi Minh', 'hcm')
]

for key in province_alias_keys:
    province_english, province_key = key
    df_province = add_province_key(df_province, province_english, province_key)

### Add alias district_keyword
There are many district was changed it's name or be combined with other districts. For instance: Quan 9 > Thanh pho Thu Duc.

Use `find_district_alias_keywords.ipynb` to create a list of tuples.

In [ ]:
df_district = df[['province_english'] + [col for col in df.columns if 'district' in col]].drop_duplicates()

district_alias_keys = [
    ('An Giang', 'Chau Thanh', 'hueduc'),
    ('Ba Ria - Vung Tau', 'Phu My', 'tanthanh'),
    ('Bac Kan', 'Ba Be', 'chora'),
    ('Bac Lieu', 'Bac Lieu', 'minhhai'),
    ('Binh Duong', 'Tan Uyen', 'chauthanh'),
    ('Binh Duong', 'Thuan An', 'laithieu'),
    ('Binh Phuoc', 'Phuoc Long', 'phuocbinh'),
    ('Ca Mau', 'Dam Doi', 'ngochien'),
    ('Cao Bang', 'Quang Hoa', 'phuchoa'),
    ('Cao Bang', 'Quang Hoa', 'quanguyen'),
    ('Cao Bang', 'Ha Quang', 'thongnong'),
    ('Cao Bang', 'Trung Khanh', 'tralinh'),
    ('Dien Bien', 'Muong Lay', 'laichau'),
    ('Dien Bien', 'Muong Cha', 'muonglay'),
    ('Dong Nai', 'Vinh Cuu', 'vinhan'),
    ('Dong Thap', 'Lap Vo', 'thanhhung'),
    ('Ha Nam', 'Phu Ly', 'hanam'),
    ('Ha Noi', 'Soc Son', 'daphuc'),
    ('Ha Noi', 'Soc Son', 'kimanh'),
    ('Hai Phong', 'Cat Hai', 'catba'),
    ('Hau Giang', 'Vi Thanh', 'mythanh'),
    ('Hau Giang', 'Nga Bay', 'tanhiep'),
    ('Hoa Binh', 'Hoa Binh', 'kyson'),
    ('Khanh Hoa', 'Dien Khanh', 'khanhxuong'),
    ('Kien Giang', 'Kien Luong', 'hatien'),
    ('Lao Cai', 'Lao Cai', 'camduong'),
    ('Nam Dinh', 'Nam Dinh', 'myloc'),
    ('Ninh Binh', 'Hoa Lu', 'giakhanh'),
    ('Ninh Binh', 'Nho Quan', 'hoanglong'),
    ('Ninh Binh', 'Yen Mo', 'tamdiep'),
    ('Phu Tho', 'Cam Khe', 'songthao'),
    ('Quang Nam', 'Nam Giang', 'giang'),
    ('Quang Ngai', 'Tra Bong', 'taytra'),
    ('Quang Ninh', 'Ha Long', 'honggai'),
    ('Quang Ninh', 'Van Don', 'campha'),
    ('Quang Ninh', 'Mong Cai', 'haininh'),
    ('Quang Ninh', 'Quang Yen', 'yenhung'),
    ('Quang Ninh', 'Ha Long', 'hoanhbo'),
    ('Tay Ninh', 'Hoa Thanh', 'phukhuong'),
    ('Thanh Hoa', 'Dong Son', 'dongthieu'),
    ('Thanh Hoa', 'Yen Dinh', 'thieuyen'),
    ('Thanh Hoa', 'Nghi Son', 'tinhgia'),
    ('Ho Chi Minh', 'Can Gio', 'duyenhai'),
    ('Ho Chi Minh', 'Thu Duc', 'quan2'),
    ('Ho Chi Minh', 'Thu Duc', 'quan9'),
    # ('Ho Chi Minh', 'Thu Duc', 'thuduc(quan)'),
    ('Tra Vinh', 'Cang Long', 'chauthanhdong'),
    ('Vinh Long', 'Long Ho', 'cainhum'),
    ('Vinh Long', 'Long Ho', 'chauthanhtay'),
    ('Ben Tre', 'Mo Cay Nam', 'mocay'),
    ('Binh Thuan', 'Ham Thuan Nam', 'hamthuan'),
    ('Ha Noi', 'Nam Tu Liem', 'tuliem'),
    ('Quang Nam', 'Nam Tra My', 'tramy'),
    ('Tien Giang', 'Go Cong Tay', 'gocong')

]

for key in district_alias_keys:
    province_english, district_english, district_key = key
    df_district = add_district_key(df_district, province_english, district_english, district_key)

Some districts contain "quan" with number in their keywords. We need to make a copy and replace "quan" to "district".

In [ ]:
hcm_districts = df_district[df_district['district_key'].str.contains(r'quan\d{1,2}')].copy()
hcm_districts['district_key'] = hcm_districts['district_key'].str.replace('quan', 'district')
df_district = pd.concat([df_district, hcm_districts])
hcm_districts['district_key'] = hcm_districts['district_key'].str.replace('district', 'q.')
df_district = pd.concat([df_district, hcm_districts])

### Create alias ward_key
Some wards contain "phuong" with number in their keywords. We need to make a copy and replace "phuong" to "ward".


In [ ]:
df_ward = df[['province_english', 'district_english'] + [col for col in df.columns if 'ward' in col]].drop_duplicates()

In [ ]:
number_wards = df_ward[df_ward['ward_key'].fillna('').str.contains(r'phuong\d{1,2}')].copy()
number_wards['ward_key'] = number_wards['ward_key'].str.replace('phuong', 'ward')
df_ward = pd.concat([df_ward, number_wards])
number_wards['ward_key'] = number_wards['ward_key'].str.replace('ward', 'p.')
df_ward = pd.concat([df_ward, number_wards])

### Create province_keys lists
We will prioritize provinces that are not in district_key or ward_key when searching a province_key in address.

In [ ]:
province_keys = df_province['province_key'].tolist()

# Pickle
province_keys_1 = []
province_keys_2 = []
province_keys_3 = []

district_keys = str(df.district_key.unique().tolist())
ward_keys = str(df.ward_key.unique().tolist())
for province_key in province_keys:
    if (province_key not in district_keys) and (province_key not in ward_keys):
        province_keys_1.append(province_key)
    elif province_key not in ward_keys:
        province_keys_2.append(province_key)
    else:
        province_keys_3.append(province_key)

### Create mapping dictionaries

In [ ]:
# Pickle
province_map = {}

for province_key in df_province.province_key.unique():
    province = df_province[df_province['province_key'] == province_key]
    province_record = province.to_dict(orient='records')[0]
    province_map[province_key] = province_record

In [ ]:
print(province_map)

In [ ]:
# Pickle
district_map = {}

for province_english in df_province.province_english.unique():
    district_keys = {}
    for district_key in df_district[df_district.province_english == province_english]['district_key'].unique():
        district_levels = {}
        for district_level_english in df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key)]['district_level_english'].unique():
            district = df_district[(df_district.province_english == province_english) & (df_district.district_key == district_key) & (df_district.district_level_english == district_level_english)]
            district_record = district[[col for col in district.columns if 'district' in col]].to_dict('records')[0]
            district_levels[district_level_english] = district_record
        district_keys[district_key] = district_levels
        
    district_keys = dict(sorted(district_keys.items(), key=lambda item: len(item[0]), reverse=True))
    
    district_map[province_english] = district_keys

In [ ]:
print(district_map['Ho Chi Minh'])

In [ ]:
# Pickle
ward_map = {}

for province_english in df_province.province_english.unique():
    districts = {}
    for district_english in df_district[df_district.province_english==province_english].district_english.unique():
        wards = {}
        for ward_key in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english)].ward_key.dropna().unique():
            ward_levels = {}
            for ward_level_english in df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key)].ward_level_english.unique():
                ward = df_ward[(df_ward.province_english==province_english) & (df_ward.district_english==district_english) & (df_ward.ward_key==ward_key) & (df_ward.ward_level_english==ward_level_english)]
                ward_record = ward[[col for col in ward.columns if 'ward' in col]].to_dict('records')[0]
                ward_levels[ward_level_english] = ward_record
            wards[ward_key] = ward_levels
        wards = dict(sorted(wards.items(), key=lambda item: len(str(item[0])), reverse=True))
        districts[district_english] = wards
    ward_map[province_english] = districts

In [ ]:
print(ward_map['Ho Chi Minh']['Tan Binh'])

### Create double check dictionaries
Some address will cause wrong mapping because they have ward_key is as same as other province's province_key.

In [ ]:
double_check_provinces = {
    'angiang': ['thainguyen', 'ninhbinh', 'thanhhoa'],
    'hoabinh': ['thainguyen',
    'langson',
    'haiphong',
    'thaibinh',
    'dongnai',
    'vinhlong',
    'dongthap'],
    'binhthuan': ['thainguyen', 'quangngai', 'binhdinh'],
    'thanhhoa': ['langson', 'binhphuoc', 'travinh', 'kiengiang', 'haugiang'],
    'thaibinh': ['langson', 'tayninh'],
    'hagiang': ['thaibinh', 'thanhhoa'],
    'quangbinh': ['thaibinh', 'thanhhoa'],
    'sonla': ['ninhbinh', 'quangngai', 'khanhhoa'],
    'khanhhoa': ['ninhbinh', 'kiengiang'],
    'longan': ['thanhhoa', 'dongnai', 'vinhlong'],
    'quangninh': ['thanhhoa'],
    'vinhlong': ['quangtri'],
    'haiduong': ['quangtri'],
    'haiphong': ['quangtri'],
    'binhduong': ['quangnam', 'quangngai', 'binhdinh'],
    'binhdinh': ['quangnam'],
    'hungyen': ['kiengiang']
}

# double_check_provinces = {} # testing

Some address will cause wrong mapping because they have ward_key is as same as other district's district_key.

In [ ]:
double_check_districts = {'unghoa': ['caugiay', 'chuongmy'],
 'thanhtri': ['hoangmai'],
 'thanhxuan': ['socson'],
 'thanhoai': ['thanhtri'],
 'thongnong': ['haquang'],
 'tralinh': ['trungkhanh'],
 'quanguyen': ['quanghoa'],
 'chora': ['babe'],
 'yenson': ['sonduong'],
 'camduong': ['laocai'],
 'muongla': ['phuyen', 'songma', 'sopcop'],
 'tranyen': ['lucyen', 'yenbinh'],
 'kyson': ['hoabinh', 'tanky'],
 'chilang': ['langson', 'trangdinh'],
 'honggai': ['halong'],
 'hoanhbo': ['halong'],
 'uongbi': ['mongcai'],
 'halong': ['vandon'],
 'haiduong': ['binhgiang'],
 'anduong': ['lechan', 'duongkinh', 'thuynguyen', 'vinhbao'],
 'catba': ['cathai'],
 'phucu': ['hungyen'],
 'myloc': ['namdinh'],
 'dongson': ['thanhhoa', 'bimson'],
 'hatrung': ['bathuoc'],
 'thanhhoa': ['nhuxuan', 'benluc'],
 'vinh': ['anhson', 'yenthanh', 'hoangmai'],
 'yenthanh': ['thanhchuong'],
 'anhson': ['thanhchuong', 'namdan'],
 'thachha': ['hatinh'],
 'huongthuy': ['hue'],
 'giang': ['thangbinh', 'bactramy', 'nuithanh'],
 'tramy': ['bactramy'],
 'sontinh': ['sontay'],
 'ducpho': ['moduc'],
 'dakto': ['konray', 'tumorong'],
 'dakha': ['tumorong'],
 'dakpo': ['kongchro'],
 'iapa': ['chuse'],
 'krongbuk': ['krongpac'],
 'phuocbinh': ['phuoclong'],
 'hoathanh': ['chauthanh'],
 'laithieu': ['thuanan'],
 'thongnhat': ['bienhoa'],
 'vinhan': ['vinhcuu'],
 'binhchanh': ['thuduc'],
 'tanphu': ['thuduc', 'quan7', 'cuchi'],
 'tanthanh': ['mochoa', 'thuthua'],
 'tanan': ['canduoc'],
 'tanhung': ['chauthanh'],
 'tanphuoc': ['gocongdong'],
 'mocay': ['mocaynam'],
 'thanhphu': ['giongtrom', 'binhdai'],
 'cainhum': ['mangthit'],
 'longho': ['mangthit'],
 'chauphu': ['chaudoc'],
 'anphu': ['phutan', 'tinhbien', 'chauthanh', 'thoaison'],
 'anminh': ['chauthanh', 'uminhthuong'],
 'vithanh': ['vithuy']}

# double_check_districts = {} # tesing

## Support find province from unique districts

Điều kiện 1: district_key not in ward_keys.
Điều kiện 2: district_count = 1, sau khi đã drop_duplicated [province_key, district_key]
Điều kiện 3: district_key not in province_keys của tỉnh khác.

Nếu district_key = province_key của chính nó:
>> Lấy danh sách long_district_alphanumerices (bao gồm Việt và English), nếu tồn tại trong address thì sau khi tìm được province_key sẽ không bị xóa province_key trong địa chỉ.


Nếu district_key != province_key của chính nó, tức là unique tuyệt đối:
>> Lấy danh sách unique_district_keys, nếu không tìm được province_key thì tìm unique_district_key để suy ngược ra province_key

In [ ]:
df = pd.read_csv('../../data/output/vietnam_administrative_units.csv')
df['province_key'] = df['province'].apply(lambda x: to_key(x, 1))
df['district_key'] = df['short_district'].apply(lambda x: to_key(x, 2))
df['ward_key'] = df['short_ward'].apply(lambda x: to_key(x, 3))

province_keys = df['province_key'].unique().tolist()
district_keys = df['district_key'].unique().tolist()
ward_keys = df['ward_key'].unique().tolist()

df_district_filter = df[~df.district_key.isin(ward_keys)]
df_district_filter_count = df_district_filter[['province_key', 'district_key']].drop_duplicates()['district_key'].value_counts().reset_index()
one_district_keys = df_district_filter_count[df_district_filter_count['count']==1]['district_key'].tolist()
df_district_filter = df_district_filter[df_district_filter.district_key.isin(one_district_keys)][['province_key', 'district_key', 'long_district', 'long_district_english']].drop_duplicates()

def check_valid_district_key(district_key, province_key):
    if district_key in province_keys and district_key != province_key:
        return False
    else:
        return True
    
df_district_filter['is_valid_district_key'] = df_district_filter.apply(lambda x: check_valid_district_key(x['district_key'], x['province_key']), axis=1)
df_district_filter = df_district_filter[df_district_filter.is_valid_district_key]
df_district_filter['same_key'] = np.where(df_district_filter.province_key == df_district_filter.district_key, True, False)

long_district_alphanumerics = []
for row in df_district_filter[df_district_filter.same_key].itertuples():
    long_district_alphanumerics.append(to_alphanumeric(row.long_district))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district).replace('thanhpho', 'tp.'))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district_english))
    
    
unique_district_keys = {}
for row in df_district_filter[~df_district_filter.same_key][['province_key', 'district_key']].drop_duplicates().itertuples():
    unique_district_keys[row.district_key] = row.province_key
    district_key = row.district_key
    if re.search(r'^quan\d{1,2}', district_key):
        district_key = re.sub('^quan', 'q.', district_key)
        unique_district_keys[district_key] = row.province_key

In [ ]:
len(long_district_alphanumerics)

In [ ]:
long_district_alphanumerics

In [ ]:
len(unique_district_keys)

In [ ]:
unique_district_keys

## Support find province from unique "long" districts

In [ ]:
df['province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['district_alphanumeric_english'] = df['long_district_english'].apply(to_alphanumeric)
df['ward_alphanumeric'] = df['long_ward'].apply(to_alphanumeric)

In [ ]:
province_alphanumerics = df['province_alphanumeric'].unique().tolist()
district_alphanumerics = df['district_alphanumeric'].unique().tolist()
ward_alphanumerics = df['ward_alphanumeric'].unique().tolist()

In [ ]:
df_district_filter = df[~df.district_alphanumeric.isin(ward_alphanumerics)]
df_district_filter_count = df_district_filter[['province_alphanumeric', 'district_alphanumeric']].drop_duplicates()['district_alphanumeric'].value_counts().reset_index()
one_district_alphanumerics = df_district_filter_count[df_district_filter_count['count']==1]['district_alphanumeric'].tolist()
df_district_filter = df_district_filter[df_district_filter.district_alphanumeric.isin(one_district_alphanumerics)][['province_key','province_alphanumeric', 'district_alphanumeric', 'long_district', 'long_district_english', 'district_key', 'district_alphanumeric_english']].drop_duplicates()

def check_valid_district_alphanumeric(district_alphanumeric, province_alphanumeric):
    if district_alphanumeric in province_alphanumerics and district_alphanumeric != province_alphanumeric:
        return False
    else:
        return True
    
df_district_filter['is_valid_district_alphanumeric'] = df_district_filter.apply(lambda x: check_valid_district_alphanumeric(x['district_alphanumeric'], x['province_alphanumeric']), axis=1)
df_district_filter = df_district_filter[df_district_filter.is_valid_district_alphanumeric]
df_district_filter['same_alphanumeric'] = np.where(df_district_filter.province_alphanumeric == df_district_filter.district_alphanumeric, True, False)

for row in df_district_filter[df_district_filter.same_alphanumeric].itertuples():
    long_district_alphanumerics.append(to_alphanumeric(row.long_district))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district).replace('thanhpho', 'tp.'))
    long_district_alphanumerics.append(to_alphanumeric(row.long_district_english))
    
    
for row in df_district_filter[~df_district_filter.same_alphanumeric][['province_key', 'district_alphanumeric', 'district_key', 'district_alphanumeric_english']].drop_duplicates().itertuples():
    if not row.district_key in unique_district_keys:
        unique_district_keys[row.district_alphanumeric] = row.province_key
        unique_district_keys[row.district_alphanumeric_english] = row.province_key
        district_alphanumeric = row.district_alphanumeric
        district_alphanumeric = re.sub('^quan', 'q.', district_alphanumeric)
        district_alphanumeric = re.sub('^huyen', 'h.', district_alphanumeric)
        district_alphanumeric = re.sub('^thanhpho', 'tp.', district_alphanumeric)
        district_alphanumeric = re.sub('^thixa', 'tx.', district_alphanumeric)
        unique_district_keys[district_alphanumeric] = row.province_key

In [ ]:
len(unique_district_keys)

In [ ]:
len(long_district_alphanumerics)

In [ ]:
print(unique_district_keys)

In [ ]:
with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump((duplicated_district_keys, duplicated_district_province_keys, duplicated_ward_keys, duplicated_ward_district_keys, province_keys_1, province_keys_2, province_keys_3, province_map, district_map, ward_map, double_check_provinces, double_check_districts, half_district_keys, long_district_alphanumerics, unique_district_keys), f)